In [1]:
import os

In [2]:
%pwd

'c:\\Users\\kafub\\Dropbox\\PC\\Desktop\\Python_and ML_BootCamp_from _Udemy\\Computer Vision\\Chest_Dis_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\kafub\\Dropbox\\PC\\Desktop\\Python_and ML_BootCamp_from _Udemy\\Computer Vision\\Chest_Dis_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionEntityConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Read constants and commns 

In [6]:
import src.Classifier.constants as constants
print(dir(constants))

['CONFIG_FILE_PATH', 'PARAMS_FILE_PATH', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [9]:
import src.Classifier.utils.common as common
print(dir(common))

['Any', 'BoxValueError', 'ConfigBox', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'base64', 'create_directories', 'decodeImage', 'encodeImageIntoBase64', 'ensure_annotations', 'get_size', 'joblib', 'json', 'load_bin', 'load_json', 'logger', 'os', 'read_yaml', 'save_bin', 'save_json', 'yaml']


In [10]:
from src.Classifier.constants import *
from src.Classifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionEntityConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionEntityConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    

In [12]:
import os
import zipfile
import gdown
from src.Classifier.logging import logger
from src.Classifier.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionEntityConfig):
        self.config = config

     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-06 18:09:04,548:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-10-06 18:09:04,548:INFO:common:yaml file: params.yaml loaded successfully]
[2024-10-06 18:09:04,548:INFO:common:created directory at: artifacts]
[2024-10-06 18:09:04,548:INFO:common:created directory at: artifacts/data_ingestion]
[2024-10-06 18:09:04,555:INFO:4161400922:Downloading data from https://drive.google.com/file/d/1NKQlHd_g5xD3lzvpXdnOswDR04grqsXE/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1NKQlHd_g5xD3lzvpXdnOswDR04grqsXE
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1NKQlHd_g5xD3lzvpXdnOswDR04grqsXE&confirm=t&uuid=021bce36-7b36-4a12-b562-9ace0e9e3b4c
To: c:\Users\kafub\Dropbox\PC\Desktop\Python_and ML_BootCamp_from _Udemy\Computer Vision\Chest_Dis_Classification\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [00:02<00:00, 59.1MB/s] 


[2024-10-06 18:09:10,430:INFO:4161400922:Downloaded data from https://drive.google.com/file/d/1NKQlHd_g5xD3lzvpXdnOswDR04grqsXE/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
